In [ ]:
import cv2
import os
import pandas as pd
from PIL import Image
import numpy as np
import re
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
img_dir = '/content/drive/MyDrive/data'
search_path = os.path.join(img_dir, "**",  "*.png")
files = glob.glob(search_path, recursive=True)
img_files = [f.replace('\\', '/') for f in files]
print(len(img_files))

9712


In [ ]:
def pad_image(img_arr, height, width):
    # Calculate the amount of padding needed
    h, w = img_arr.shape[:2]
    pad_h = max(0, height - h)
    pad_w = max(0, width - w)

    # Add padding to the image
    padded_arr = np.pad(img_arr, ((0, pad_h), (0, pad_w)), mode='constant', constant_values=0)
    return(padded_arr)


In [ ]:
filename = []
maxh = 310
maxw = 360
image_data = np.zeros((len(img_files), maxh, maxw),dtype=np.uint8)
shape = []

for i,img_path in enumerate(img_files):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) # Convert to grayscale
    img_np = np.asarray(img)
    label1 = re.search(r'case[\d]+_day[\d]+', img_path)
    label2 = re.search(r'slice_[\d]+', img_path)
    tmp_list = img_path.split('_')
    shape.append((int(tmp_list[4]), int(tmp_list[3])))
    fn = label1[0]+"_"+label2[0]
    image_data[i] = pad_image(img_np, maxh, maxw)
    filename.append(fn)
    if (i%100 == 0):
      print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200


KeyboardInterrupt: ignored

In [ ]:
print(filename[0])
print(len(filename))
print(shape)


case92_day0_slice_0026
2240
[(266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266), (266, 266),

In [ ]:
img2_5D = np.zeros((len(img_files), 3, maxh, maxw), dtype = int)
labels = np.zeros((len(img_files), 3, maxh, maxw), dtype = int)
valid_seg = np.zeros(len(img_files), dtype=bool)

In [ ]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    if not isinstance(mask_rle, str):
        img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
        return img.reshape(shape)

    s = mask_rle.split()
    s = s[1:]
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/data.csv')
pd.options.display.max_colwidth = None
for i in range(len(img_files)):
  # process the img2.5D
  if (i == 0 or i == len(img_files) - 1):
    continue
  else:
    pre_sn = re.search(r"(\w+_\w+)_slice", filename[i-1]).group(1)
    curr_sn = re.search(r"(\w+_\w+)_slice", filename[i]).group(1)
    next_sn = re.search(r"(\w+_\w+)_slice", filename[i+1]).group(1)
    img2_5D[i,1,:,:] = image_data[i]

    if(pre_sn.lower() == curr_sn.lower()):
      img2_5D[i,0,:,:] = image_data[i-1]
    else:
      continue

    if(next_sn.lower() == curr_sn.lower()):
      img2_5D[i,2,:,:] = image_data[i + 1]
    else:
      continue

  # process the masks aka labels
  fn = filename[i]
  filtered_df = df[(df['id'] == fn) & (df['class'] == 'large_bowel')]
  if (filtered_df["segmentation"].isna().all()):
    continue
  labels[i,0,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)

  filtered_df = df[(df['id'] == fn) & (df['class'] == 'small_bowel')]
  if (filtered_df["segmentation"].isna().all()):
    continue
  labels[i,1,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)

  filtered_df = df[(df['id'] == fn) & (df['class'] == 'stomach')]
  if (filtered_df["segmentation"].isna().all()):
    continue
  labels[i,2,:,:] = pad_image(rle_decode(filtered_df["segmentation"].to_string(), shape[i]), maxh, maxw)
  valid_seg[i] = 1

In [ ]:
print(valid_seg)

[False False False ... False  True False]


In [ ]:
res_img = img2_5D[valid_seg, :,:,:]
res_label = labels[valid_seg, :,:,:]

In [ ]:
print(res_label.shape)
print(res_img.shape)

(172, 3, 310, 360)
(172, 3, 310, 360)


In [ ]:
np.save("/content/drive/MyDrive/EECS545 Final Project/images_25D_new.npy", res_img)

In [ ]:
np.save("/content/drive/MyDrive/EECS545 Final Project/full_seg_labels_new.npy", res_label)